In [ ]:
from datasets import load_from_disk
import os
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from typing import List, Dict, Callable, Optional
import numpy as np
import json
import tqdm
from functools import partial

# LlamaIndex 관련
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import QueryFusionRetriever, BaseRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.schema import TextNode, NodeWithScore, QueryBundle, BaseNode
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# 기타 라이브러리
import bm25s
from gliner import GLiNER
from sentence_transformers import CrossEncoder 

# Hugging Face 로그인
print("🚀 환경 설정 시작!")
HF_TOKEN = ""
os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN 
login(token=HF_TOKEN)

print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

🚀 환경 설정 시작!
GPU 사용 가능 여부: True


In [2]:
# wiki data load
with open('/data/ephemeral/home/data/wikipedia_documents.json') as f:
    wiki_data = json.load(f)
id_to_title = {v["document_id"]: v["title"] for v in wiki_data.values()}


train_set_dir = "/data/ephemeral/home/data/train_dataset/"
dataset = load_from_disk(train_set_dir)

In [3]:
# --- 모델 로드 설정 ---
GEMMA_MODEL_NAME = "google/gemma-3-4b-it"  # 메모리 효율성을 위해 4b 대신 9b를 예시로 사용 (사용자 환경에 따라 변경 가능)
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
RERANKER_MODEL_NAME = "BAAI/bge-reranker-v2-m3"

In [4]:
# --- LLM 및 Tokenizer 로드 ---
def load_gemma():
    """Gemma 모델과 토크나이저를 로드합니다."""
    # Q: Gemma 3-4b-it 사용 예정이었는데, 현재는 Gemma 2-9b-it을 사용하려 합니다.
    # A: VRAM 상황에 따라 모델 이름을 적절히 변경하여 사용하세요.
    tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        GEMMA_MODEL_NAME,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        torch_dtype=torch.bfloat16,
    )
    return tokenizer, model

In [5]:
documents: List[Document] = []
for doc_id, data in wiki_data.items():
    # 'text' 필드를 문서 내용으로 사용
    documents.append(
        Document(
            text=data['text'],
            metadata={
                "document_id": data['document_id'],
                "title": data['title'],
                "corpus_source": data['corpus_source']
            }
        )
    )

In [6]:
# 3. 문서 청킹 (Node 생성)
# SentenceSplitter는 문장 단위 분할을 기본으로 하면서, 
# 최종 청크 크기를 chunk_size=512로 제한합니다.
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=50)

# nodes에는 작은 텍스트 청크(TextNode)들이 리스트 형태로 담깁니다.
nodes: List[TextNode] = splitter.get_nodes_from_documents(documents)

print(f"원본 문서 개수: {len(documents)}개")
print(f"생성된 청크(Node) 개수: {len(nodes)}개")
print(f"첫 번째 청크 텍스트 예시: {nodes[0].get_content()[:100]}...")

원본 문서 개수: 60613개
생성된 청크(Node) 개수: 128727개
첫 번째 청크 텍스트 예시: 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부...


In [7]:
torch.cuda.is_available() 

True

In [8]:
embed_model = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    device="cuda" if torch.cuda.is_available() else "cpu"
)



In [9]:
print("--- VectorStoreIndex 생성 시작 (임베딩 중) ---")
#vector_index = VectorStoreIndex(nodes, embed_model=embed_model)


--- VectorStoreIndex 생성 시작 (임베딩 중) ---


In [10]:
# faiss vs

dummy_emb = embed_model.get_text_embedding("dim 체크용")
dim = len(dummy_emb)
faiss_index = faiss.IndexFlatIP(dim) 
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [11]:
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
)

In [13]:
# --- 3. Reranker (사용자 정의) ---
from sentence_transformers import CrossEncoder 

class Reranker:
    def __init__(self, model_name: str = RERANKER_MODEL_NAME):
        self.model = CrossEncoder(model_name, device="cuda" if torch.cuda.is_available() else "cpu")

    def rerank(self, query: str, docs: List[Dict], doc_id, top_k: int = 5) -> List[Dict]:
        """
        query와 docs[{'text': ..., ...}]를 받아, score 기준으로 다시 정렬해서 top_k만 반환합니다.
        """
        if not docs:
            return []

        pairs = [[query, d] for d in docs]
        scores = self.model.predict(pairs)  # shape (len(docs),)
        scored_docs = list(zip(docs, scores))
        scored_id = list(zip(doc_id, scores)) 

        scored_docs.sort(key=lambda x: x[1], reverse=True)
        scored_id.sort(key=lambda x: x[1], reverse=True)

        return scored_docs[:top_k], scored_id[:top_k]

In [14]:
reranker = Reranker()


In [15]:
tokenizer, model = load_gemma()
gemma_llm = HuggingFaceLLM(
    # model_name을 지정할 필요가 없거나, 명시적으로 지정해도 model/tokenizer 인자가 우선됩니다.
    model=model,        # 이미 로드된 PyTorch 모델 객체
    tokenizer=tokenizer,  # 이미 로드된 Tokenizer 객체
    #device_map=="cuda" if torch.cuda.is_available() else "cpu",
    # device_map="auto" 등은 이미 model 로드 시 적용되었으므로 LlamaIndex LLM에서는 불필요
    
    # 템플릿 처리 방식 등 LlamaIndex 관련 설정만 추가
    context_window=8192, # 예시: Gemma의 Context Window 설정 (필요에 따라)
)

# 3. LlamaIndex 설정에 적용
Settings.llm = gemma_llm

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# import gc
# import torch

# print(f"🧹 청소 전 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

# # 1. LLM (Gemma) 방 빼!
# if 'gemma_llm' in locals(): del gemma_llm
# if 'model' in locals(): del model  # LLM 원본 모델 객체
# if 'tokenizer' in locals(): del tokenizer # LLM 토크나이저

# # 2. Reranker 방 빼!
# if 'reranker' in locals(): del reranker

# # 3. Embedding Model 방 빼! (벡터 인덱스 생성 끝났다면)
# if 'embed_model' in locals(): del embed_model

# # 4. 강제 수거 (Garbage Collection)
# gc.collect()
# torch.cuda.empty_cache()

# print(f"✨ 청소 후 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
# print("   -> 이제 GLiNer가 운동장을 혼자 쓸 수 있습니다!")

🧹 청소 전 메모리: 19.93 GB
✨ 청소 후 메모리: 17.82 GB
   -> 이제 GLiNer가 운동장을 혼자 쓸 수 있습니다!


In [16]:
print("🔥 GLiNer XL 모델 로딩 중...")

# 1. 라벨 정의
entity_type_mapping = {
    "PS": { "PS_NAME": "인물_사람", "PS_CHARACTER": "인물_가상 캐릭터", "PS_PET": "인물_반려동물"},
    "FD": { "FD_SCIENCE": "학문 분야_과학", "FD_SOCIAL_SCIENCE": "학문 분야_사회과학", "FD_MEDICINE": "학문 분야_의학", "FD_ART": "학문 분야_예술", "FD_HUMANITIES": "학문 분야_인문학", "FD_OTHERS": "학문 분야_기타"},
    "TR": { "TR_SCIENCE": "이론_과학", "TR_SOCIAL_SCIENCE": "이론_사회과학", "TR_MEDICINE": "이론_의학", "TR_ART": "이론_예술", "TR_HUMANITIES": "이론_철학/언어/역사", "TR_OTHERS": "이론_기타"},
    "AF": { "AF_BUILDING": "인공물_건축물/토목건설물", "AF_CULTURAL_ASSET": "인공물_문화재", "AF_ROAD": "인공물_도로/철로", "AF_TRANSPORT": "인공물_교통수단/운송수단", "AF_MUSICAL_INSTRUMENT": "인공물_악기", "AF_WEAPON": "인공물_무기", "AFA_DOCUMENT": "인공물_도서/서적 작품명", "AFA_PERFORMANCE": "인공물_춤/공연/연극 작품명", "AFA_VIDEO": "인공물_영화/TV 프로그램", "AFA_ART_CRAFT": "인공물_미술/조형 작품명", "AFA_MUSIC": "인공물_음악 작품명", "AFW_SERVICE_PRODUCTS": "인공물_서비스 상품", "AFW_OTHER_PRODUCTS": "인공물_기타 상품"},
    "OG": { "OGG_ECONOMY": "기관_경제", "OGG_EDUCATION": "기관_교육", "OGG_MILITARY": "기관_군사", "OGG_MEDIA": "기관_미디어", "OGG_SPORTS": "기관_스포츠", "OGG_ART": "기관_예술", "OGG_MEDICINE": "기관_의료", "OGG_RELIGION": "기관_종교", "OGG_SCIENCE": "기관_과학", "OGG_LIBRARY": "기관_도서관", "OGG_LAW": "기관_법률", "OGG_POLITICS": "기관_정부/공공", "OGG_FOOD": "기관_음식 업체", "OGG_HOTEL": "기관_숙박 업체", "OGG_OTHERS": "기관_기타"},
    "LC": { "LCP_COUNTRY": "장소_국가", "LCP_PROVINCE": "장소_도/주 지역", "LCP_COUNTY": "장소_세부 행정구역", "LCP_CITY": "장소_도시", "LCP_CAPITALCITY": "장소_수도", "LCG_RIVER": "장소_강/호수", "LCG_OCEAN": "장소_바다", "LCG_BAY": "장소_반도/만", "LCG_MOUNTAIN": "장소_산/산맥", "LCG_ISLAND": "장소_섬", "LCG_CONTINENT": "장소_대륙", "LC_SPACE": "장소_천체", "LC_OTHERS": "장소_기타"},
    "CV": { "CV_CULTURE": "문명_문명/문화", "CV_TRIBE": "문명_민족/종족", "CV_LANGUAGE": "문명_언어", "CV_POLICY": "문명_제도/정책", "CV_LAW": "문명_법/법률", "CV_CURRENCY": "문명_통화", "CV_TAX": "문명_조세", "CV_FUNDS": "문명_연금/기금", "CV_ART": "문명_예술", "CV_SPORTS": "문명_스포츠", "CV_SPORTS_POSITION": "문명_스포츠 포지션", "CV_SPORTS_INST": "문명_스포츠 용품/도구", "CV_PRIZE": "문명_상/훈장", "CV_RELATION": "문명_가족/친족 관계", "CV_OCCUPATION": "문명_직업", "CV_POSITION": "문명_직위/직책", "CV_FOOD": "문명_음식", "CV_DRINK": "문명_음료/술", "CV_FOOD_STYLE": "문명_음식 유형", "CV_CLOTHING": "문명_의복/섬유", "CV_BUILDING_TYPE": "문명_건축 양식"},
    "DT": { "DT_DURATION": "날짜_기간", "DT_DAY": "날짜_일", "DT_WEEK": "날짜_주(주차)", "DT_MONTH": "날짜_달(월)", "DT_YEAR": "날짜_연(년)", "DT_SEASON": "날짜_계절", "DT_GEOAGE": "날짜_지질시대", "DT_DYNASTY": "날짜_왕조시대", "DT_OTHERS": "날짜_기타"},
    "TI": { "TI_DURATION": "시간_기간", "TI_HOUR": "시간_시각(시)", "TI_MINUTE": "시간_분", "TI_SECOND": "시간_초", "TI_OTHERS": "시간_기타"},
    "QT": { "QT_AGE": "수량_나이", "QT_SIZE": "수량_넓이/면적", "QT_LENGTH": "수량_길이/거리", "QT_COUNT": "수량_수량/빈도", "QT_MAN_COUNT": "수량_인원수", "QT_WEIGHT": "수량_무게", "QT_PERCENTAGE": "수량_백분율", "QT_SPEED": "수량_속도", "QT_TEMPERATURE": "수량_온도", "QT_VOLUME": "수량_부피", "QT_ORDER": "수량_순서", "QT_PRICE": "수량_금액", "QT_PHONE": "수량_전화번호", "QT_SPORTS": "수량_스포츠 수량", "QT_CHANNEL": "수량_채널 번호", "QT_ALBUM": "수량_앨범 수량", "QT_ADDRESS": "수량_주소 관련 숫자", "QT_OTHERS": "수량_기타 수량"},
    "EV": { "EV_ACTIVITY": "사건_사회운동/선언", "EV_WAR_REVOLUTION": "사건_전쟁/혁명", "EV_SPORTS": "사건_스포츠 행사", "EV_FESTIVAL": "사건_축제/영화제", "EV_OTHERS": "사건_기타"},
    "AM": { "AM_INSECT": "동물_곤충", "AM_BIRD": "동물_조류", "AM_FISH": "동물_어류", "AM_MAMMALIA": "동물_포유류", "AM_AMPHIBIA": "동물_양서류", "AM_REPTILIA": "동물_파충류", "AM_TYPE": "동물_분류명", "AM_PART": "동물_부위명", "AM_OTHERS": "동물_기타"},
    "PT": { "PT_FRUIT": "식물_과일/열매", "PT_FLOWER": "식물_꽃", "PT_TREE": "식물_나무", "PT_GRASS": "식물_풀", "PT_TYPE": "식물_분류명", "PT_PART": "식물_부위명", "PT_OTHERS": "식물_기타"},
    "MT": { "MT_ELEMENT": "물질_원소", "MT_METAL": "물질_금속", "MT_ROCK": "물질_암석", "MT_CHEMICAL": "물질_화학"},
    "TM": { "TM_COLOR": "용어_색깔", "TM_DIRECTION": "용어_방향", "TM_CLIMATE": "용어_기후 지역", "TM_SHAPE": "용어_모양/형태", "TM_CELL_TISSUE_ORGAN": "용어_세포/조직/기관", "TMM_DISEASE": "용어_증상/질병", "TMM_DRUG": "용어_약품", "TMI_HW": "용어_IT 하드웨어", "TMI_SW": "용어_IT 소프트웨어", "TMI_SITE": "용어_URL 주소", "TMI_EMAIL": "용어_이메일 주소", "TMI_MODEL": "용어_제품 모델명", "TMI_SERVICE": "용어_IT 서비스", "TMI_PROJECT": "용어_프로젝트", "TMIG_GENRE": "용어_게임 장르", "TM_SPORTS": "용어_스포츠"},
}
labels = []
for main_category in entity_type_mapping:
    sub_dict = entity_type_mapping[main_category]
    for key in sub_dict:
        labels.append(sub_dict[key])

# 2. GLiNer XL 모델 로드
gliner_model = GLiNER.from_pretrained("lots-o/gliner-bi-ko-xlarge-v1")
gliner_model = gliner_model.to("cuda")
print("✅ GLiNer XL 모델 로드 완료!")

🔥 GLiNer XL 모델 로딩 중...


/data/ephemeral/home/mcr/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

/data/ephemeral/home/mcr/.venv/lib/python3.12/site-packages/gliner/model.py:418: UserWarning: Resizing embeddings is not supported for bi-encoder models.
  instance.resize_embeddings()


✅ GLiNer XL 모델 로드 완료!


In [18]:
from functools import partial
import bm25s
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle, BaseNode
from typing import List, Dict, Callable, Optional
import tqdm

# -----------------------------------------------------------------------------
# 3. [핵심] 배치 지원 토크나이저 함수 (List[str] 입력을 받음)
# -----------------------------------------------------------------------------
def tokenize_gliner_batch(
    texts: List[str],  # 🚨 입력이 단일 str이 아니라 List[str]입니다!
    model: GLiNER, 
    labels: List[str], 
    label_chunk_size: int = 20,
    score_threshold_ratio: float = 1.05
) -> List[List[str]]:
    """
    여러 문장을 한 번에 받아서, 각 문장별 토큰 리스트를 반환합니다.
    (V100 GPU 병렬 처리를 위해 필수)
    """
    if not texts: return []

    # 결과 저장용 (문서 개수만큼 빈 리스트 생성)
    batch_results = [[] for _ in texts]
    
    # 라벨 배치 처리 (Label Chunking)
    for i in range(0, len(labels), label_chunk_size):
        sub_labels = labels[i : i + label_chunk_size]
        try:
            # 🚀 model.batch_predict_entities 사용 (속도 향상의 핵심)
            batch_preds = model.batch_predict_entities(
                texts, sub_labels, flat_ner=True, threshold=0.1
            )
            # 결과 병합
            for doc_idx, entities in enumerate(batch_preds):
                batch_results[doc_idx].extend(entities)
        except Exception: continue
    
    # 후처리 (각 문서별로 점수 필터 & 문장 쪼개기 적용)
    final_batch_tokens = []
    
    for entities in batch_results:
        if not entities:
            final_batch_tokens.append([])
            continue
            
        # 1. 점수 필터링 (Relative Threshold)
        max_score = max(e['score'] for e in entities)
        cutoff_score = max_score / score_threshold_ratio
        filtered = [e for e in entities if e['score'] >= cutoff_score]
        
        # 2. 스마트 필터 (문장 쪼개기)
        doc_tokens = set()
        for e in filtered:
            token_text = e['text']
            # 띄어쓰기 2개 이상(3어절)이면 쪼개기
            if token_text.count(' ') >= 2:
                for t in token_text.split():
                    doc_tokens.add(t)
            else:
                doc_tokens.add(token_text)
        
        final_batch_tokens.append(list(doc_tokens))
        
    return final_batch_tokens


class GLiNerBM25Retriever(BaseRetriever):
    """partial 토크나이저를 받아서 배치 처리하는 Retriever"""
    def __init__(
        self,
        nodes: List[BaseNode],
        tokenizer: Callable[[List[str]], List[List[str]]], # 🚨 배치 토크나이저 시그니처
        similarity_top_k: int = 30,
        doc_batch_size: int = 64 # 🚀 문서 배치 사이즈 (한 번에 처리할 문서 수)
    ) -> None:
        
        self._nodes = nodes
        self._similarity_top_k = similarity_top_k
        self._tokenizer = tokenizer
        self.doc_batch_size = doc_batch_size

        print(f"🚀 GLiNer Batch Indexing... Docs: {len(nodes)}, Batch: {doc_batch_size}")
        
        corpus_tokens = []
        
        # 문서를 뭉텅이(Batch)로 잘라서 토크나이저 함수 호출
        for i in tqdm.tqdm(range(0, len(nodes), doc_batch_size), desc="Indexing"):
            batch_nodes = nodes[i : i + doc_batch_size]
            batch_texts = [n.text if n.text else "" for n in batch_nodes]
            
            # 여기서 partial로 만든 함수에 '리스트'를 던집니다!
            batch_tokens_list = self._tokenizer(batch_texts)
            corpus_tokens.extend(batch_tokens_list)

        self._bm25 = bm25s.BM25()
        self._bm25.index(corpus_tokens)
        
        print("✅ Indexing Complete!")
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query = query_bundle.query_str
        
        # 쿼리는 1개지만, 배치 함수니까 리스트로 감싸서 보냄
        # 결과도 리스트의 리스트니까 [0]으로 꺼냄
        query_tokens = self._tokenizer([query])[0]
        
        if not query_tokens: return []
        
        tokenized_query = [query_tokens]
        actual_k = min(self._similarity_top_k, len(self._nodes))
        if actual_k == 0: return []

        results, scores = self._bm25.retrieve(tokenized_query, k=actual_k)
        
        nodes_with_scores: List[NodeWithScore] = []
        for idx, score in zip(results[0], scores[0]):
            if score > 0:
                nodes_with_scores.append(
                    NodeWithScore(node=self._nodes[idx], score=float(score))
                )
        return nodes_with_scores

In [20]:
# 1. 배치 토크나이저 함수를 partial로 조립
my_gliner_batch_tokenizer = partial(
    tokenize_gliner_batch, 
    model=gliner_model,         # XL 모델
    labels=labels,              # 150개 라벨
    label_chunk_size=50,        # 라벨 20개씩 끊기
    score_threshold_ratio=2.5  # 점수 필터
)

# 2. Retriever 생성 (인덱싱 시작)
# nodes는 이전 단계에서 이미 생성되어 있어야 합니다!
gliner_bm25_retriever = GLiNerBM25Retriever(
    nodes=nodes,
    similarity_top_k=30,
    tokenizer=my_gliner_batch_tokenizer, # partial 함수 전달
    doc_batch_size=8 # 🚀 한 번에 64개씩 처리 (V100 메모리 활용)
)

print("✅ GLiNer Retriever 준비 완료 (Batch Mode)")

🚀 GLiNer Batch Indexing... Docs: 128727, Batch: 8


Indexing:   0%|          | 0/16091 [00:00<?, ?it/s]/tmp/ipykernel_496380/3586281713.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(
Indexing:  72%|███████▏  | 11565/16091 [4:12:06<1:39:44,  1.32s/it]/data/ephemeral/home/mcr/.venv/lib/python3.12/site-packages/gliner/data_processing/processor.py:372: UserWarning: Sentence of length 522 has been truncated to 512
  batch = [self.preprocess_example(b["tokenized_text"], b[key], class_to_ids) for b in batch_list]
Indexing:  82%|████████▏ | 13115/16091 [4:45:42<1:02:11,  1.25s/it]/data/ephemeral/home/mcr/.venv/lib/python3.12/site-packages/gliner/data_processing/processor.py:372: UserWarning: Sentence of length 523 has been truncated to 512
  batch = [self.preprocess_example(b["tokenized_text"], b[key], class_to_ids) for b in batch_list]
Indexing: 100%|██████████| 16091/16091 [5:50:08<00:00,  1.31s/it]  


BM25S Create Vocab:   0%|          | 0/128727 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/128727 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/128727 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/128727 [00:00<?, ?it/s]

✅ Indexing Complete!
✅ GLiNer Retriever 준비 완료 (Batch Mode)


In [21]:
#골든리트리버
retriever = vector_index.as_retriever(similarity_top_k=50)



print("🔗 Fusion Retriever 연결")
fusion_retriever = QueryFusionRetriever(
    retrievers=[retriever, gliner_bm25_retriever],
    similarity_top_k=30,  
    num_queries=1,
    use_async=False,
    mode="reciprocal_rerank"
)





🔗 Fusion Retriever 연결


In [30]:
# 1. 확인할 질문 가져오기
query = dataset['train'][307]['question']

print(f"❓ 원본 질문: {query}")
print("-" * 50)

# 2. 리트리버 내부의 토크나이저 직접 호출
# (우리가 배치 처리를 위해 List[str] 입력을 받게 만들었으므로 [query] 리스트로 넣어줍니다)
tokens_batch = gliner_bm25_retriever._tokenizer([query])

# 3. 결과 확인 (배치 리스트의 첫 번째 요소)
extracted_tokens = tokens_batch[0]

print(f"🧩 GLiNer가 뽑은 토큰: {extracted_tokens}")
print("-" * 50)

# 4. 결과 해석
if not extracted_tokens:
    print("😱 [경고] 토큰이 하나도 없습니다! (검색 결과 0건 확정)")
    print("   -> GLiNer가 이 질문에서 중요한 단어를 못 찾았습니다.")
else:
    print(f"✅ 총 {len(extracted_tokens)}개의 키워드로 검색을 수행합니다.")

❓ 원본 질문: 남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?
--------------------------------------------------


/tmp/ipykernel_496380/3586281713.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


🧩 GLiNer가 뽑은 토큰: ['뉴잉글랜드', '뉴잉글랜드초원', '멧닭']
--------------------------------------------------
✅ 총 3개의 키워드로 검색을 수행합니다.


In [31]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# 2. GLiNer 단독 검색 실행
# (내부에서 print(f"🔍 Query Tokens: ...") 로그가 찍힐 겁니다. 이걸 꼭 확인하세요!)
gliner_results = gliner_bm25_retriever.retrieve(query)

# 3. 결과 뜯어보기
print(f"\n🔍 [GLiNer Only] 검색 결과: {len(gliner_results)}개 발견\n")

for i, node_with_score in enumerate(gliner_results[:5]): # 상위 5개만
    node = node_with_score.node
    score = node_with_score.score
    
    print(f"🏅 [Rank {i+1}] BM25 Score: {score:.4f}")
    print(f"📄 문서 ID: {node.metadata.get('document_id', 'N/A')}")
    # 가독성을 위해 줄바꿈 제거하고 100자만 출력
    print(f"📝 내용: {node.text[:100].replace('\n', ' ')}...") 
    print("-" * 60)

❓ 질문: 남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?
------------------------------------------------------------


/tmp/ipykernel_496380/3586281713.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


🔍 [GLiNer Only] 검색 결과: 30개 발견

🏅 [Rank 1] BM25 Score: 8.7178
📄 문서 ID: 24634
📝 내용: 1791년 일부 정치인들은 수렵시즌을 도입하려 했지만, 사람들의 비웃음만 사게 되었다. 그 결과 뉴잉글랜드초원멧닭은 다른 두 아종처럼 과도한 남획으로 인해 큰 타격을 입어 19세기...
------------------------------------------------------------
🏅 [Rank 2] BM25 Score: 6.7319
📄 문서 ID: 24634
📝 내용: 초원뇌조들은 미국 독립전쟁 이전에 버지니아 주에서부터 메인주까지 서식했다. 이들은 상당히 흔했다. 1620년에 플리머스(Plymouth, 영국의 항구 이름을 딴 미국의 마을로 현재...
------------------------------------------------------------
🏅 [Rank 3] BM25 Score: 4.7166
📄 문서 ID: 56478
📝 내용: 하지만, 뉴잉글랜드 청교도는 하버드 대학의 잔디밭에 나온 것처럼,  뉴잉글랜드의 첫열매로 하버드 대학을 상징하도록 하였다. 그들은 많은 사람들이 의외로 무지한 것에 놀랐으며, 그들...
------------------------------------------------------------
🏅 [Rank 4] BM25 Score: 4.2644
📄 문서 ID: 24635
📝 내용: 이로 인해 초원멧닭은 더욱 큰 피해를 입게 되었다. 게다가, 남은 뉴잉글랜드초원멧닭들 다수는 가축 칠면조가 이 섬에 도입한 가금류 질병인 흑두병(닭목의 동물들이 걸리는 질병으로 칠...
------------------------------------------------------------
🏅 [Rank 5] BM25 Score: 3.8913
📄 문서 ID: 47072
📝 내용: 신앙주의(fideism)란 신앙에 근거한 학문과 삶의 체계를 말한다. 보통 이성주의에 반대적 개

In [32]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}\n")

# 2. 검색 실행
retrieved_nodes = fusion_retriever.retrieve(query)

# 3. 결과 뜯어보기 (상위 5개만 출력)
print(f"🔍 총 {len(retrieved_nodes)}개의 문서를 찾았습니다.\n")

print("="*60)
for i, node_with_score in enumerate(retrieved_nodes[:5]):
    node = node_with_score.node
    score = node_with_score.score
    
    # Fusion Retriever는 여러 리트리버 결과를 섞기 때문에
    # 메타데이터를 통해 원래 어디서 왔는지(Vector? GLiNer?) 유추해야 할 수도 있지만,
    # 기본적으로 RRF(Reciprocal Rank Fusion) 점수가 출력됩니다.
    
    print(f"🏆 [Rank {i+1}] Score: {score:.4f}")
    print(f"📄 문서 ID: {node.metadata.get('document_id', 'N/A')}")
    print(f"📝 내용: {node.text[:100].replace('\n', ' ')}...") # 100자만 미리보기
    print("-" * 60)

❓ 질문: 남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?



/tmp/ipykernel_496380/3586281713.py:32: FutureWarning: GLiNER.batch_predict_entities is deprecated and will be removed in a future release. Please use GLiNER.inference instead.
  batch_preds = model.batch_predict_entities(


BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]

🔍 총 30개의 문서를 찾았습니다.

🏆 [Rank 1] Score: 0.0328
📄 문서 ID: 24634
📝 내용: 1791년 일부 정치인들은 수렵시즌을 도입하려 했지만, 사람들의 비웃음만 사게 되었다. 그 결과 뉴잉글랜드초원멧닭은 다른 두 아종처럼 과도한 남획으로 인해 큰 타격을 입어 19세기...
------------------------------------------------------------
🏆 [Rank 2] Score: 0.0323
📄 문서 ID: 24635
📝 내용: 이로 인해 초원멧닭은 더욱 큰 피해를 입게 되었다. 게다가, 남은 뉴잉글랜드초원멧닭들 다수는 가축 칠면조가 이 섬에 도입한 가금류 질병인 흑두병(닭목의 동물들이 걸리는 질병으로 칠...
------------------------------------------------------------
🏆 [Rank 3] Score: 0.0318
📄 문서 ID: 24634
📝 내용: 초원뇌조들은 미국 독립전쟁 이전에 버지니아 주에서부터 메인주까지 서식했다. 이들은 상당히 흔했다. 1620년에 플리머스(Plymouth, 영국의 항구 이름을 딴 미국의 마을로 현재...
------------------------------------------------------------
🏆 [Rank 4] Score: 0.0306
📄 문서 ID: 24635
📝 내용: 다행히도, 그때쯤 이들을 보호하자는 여론이 조성되었다. 648만 평방미터에 달하는 섬은 보호구로 지정했다. 효과는 뛰어나서 뉴잉글랜드초원멧닭의 개체수가 1915년에는 800마리, ...
------------------------------------------------------------
🏆 [Rank 5] Score: 0.0303
📄 문서 ID: 27558
📝 내용: 그는 곧 뉴잉글랜드 해안의 다수 종족인 파두셋 부족을 발견했지만, 그들은 이미 천연두로 의심되는 몇 년에 걸친 전염병으로 많은 이들이 죽은 이후였다. (2010

In [33]:
docs_for_rerank = [n.node.text for n in retrieved_nodes]
ids_for_rerank = [n.node.metadata['document_id'] for n in retrieved_nodes]

reranked_results = reranker.rerank(query, docs_for_rerank, ids_for_rerank, top_k=5)


In [34]:

final_context = "\n\n---\n\n".join([f"[{i+1}] {d[0]}" for i, d in enumerate(reranked_results[0])])
print(f'Q. {query}')

print(np.array(reranked_results[1])[:,0])

Q. 남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?
[24635. 24636. 24634. 24634. 24635.]


In [35]:
dataset['train'][307]

{'title': '뉴잉글랜드초원뇌조',
 'context': '다행히도, 그때쯤 이들을 보호하자는 여론이 조성되었다. 648만 평방미터에 달하는 섬은 보호구로 지정했다. 효과는 뛰어나서 뉴잉글랜드초원멧닭의 개체수가 1915년에는 800마리, 1916년에는 2천 마리까지 늘었다. 문제는 이 아종의 전체 개체군이 이 작은 섬 하나에만 한정적으로 서식했기 때문에 굉장히 취약했다. 게다가 이 섬의 개체군 일부를 다른 섬이나 본토에 옮기는 시도는 없었다. 즉, 이 아종 전체가 재난에 대단히 취약하다는 결론이 나오게 된다. 실제로 2,000마리까지 증가한 1916년에 섬 전역에서 일어난 화재로 번식지가 파괴되고 말았다. 공교롭게도 이 시기는 번식기여서 대부분의 초원멧닭과 알들도 불에 타 죽어갔다. 그 결과 겨우 105마리만 살아남았다. 다음 해 겨울은 매우 혹독하게 추웠으며, 초원멧닭의 천적인 참매가 이 섬에 들어오게 되었다. 이로 인해 초원멧닭은 더욱 큰 피해를 입게 되었다. 게다가, 남은 뉴잉글랜드초원멧닭들 다수는 가축 칠면조가 이 섬에 도입한 가금류 질병인 흑두병(닭목의 동물들이 걸리는 질병으로 칠면조편모충의 감염이 원인이다)에 감염되어 대량 폐사했다.',
 'question': '남은 뉴잉글랜드초원멧닭들을 떼죽음으로 몰고 간 병은?',
 'id': 'mrc-0-000315',
 'answers': {'answer_start': [505], 'text': ['흑두병']},
 'document_id': 24635,
 '__index_level_0__': 210}

In [36]:
print(final_context)

[1] 이로 인해 초원멧닭은 더욱 큰 피해를 입게 되었다. 게다가, 남은 뉴잉글랜드초원멧닭들 다수는 가축 칠면조가 이 섬에 도입한 가금류 질병인 흑두병(닭목의 동물들이 걸리는 질병으로 칠면조편모충의 감염이 원인이다)에 감염되어 대량 폐사했다.

---

[2] 그 결과 1927년에는 뉴잉글랜드초원멧닭이 겨우 13마리밖에 남지 않았고 거의 대부분은 수컷이었다. 마지막 남은 뉴잉글랜드초원멧닭을 연구한 아서 그로스(Arthur Gross)는 마지막 남은 이 새에 대한 최종 보고서에 다음과 같이 썼다. "모든 날씨 변화와 질병, 천적의 영향을 받는 이 새가 그렇게 오랫동안 홀로 살 수 있었던 것은 정말 놀랍다." 1931년 4월, 그로스는 이 홀로 남은 새를 잡아 띠를 두른 후 놓아주었다.

1932년 3월 11일 저녁에 마지막 한마리인 8살짜리 수컷이 죽으면서, 연속적으로 재난에 휩싸인 뉴잉글랜드초원멧닭은 멸종했다. 지방신문은 "생존자는 없다. 미래도 없다. 이런 종류로 재창조되는 생명은 더이상 없을 것이다."고 보도했다. 이로써 뉴잉글랜드초원뇌조는 멸종했다.

---

[3] 한때 로키산맥에서부터 애팔레치아 산맥 동부에 걸친 서식영역을 자랑하던 큰초원멧닭은 농부들과 정착민이 이들의 서식지를 파괴했고, 사냥꾼들은 이들을 총으로 쏴 20세기 초에 이 윙윙거리며 나는 큰초원뇌조는 멸종위기에 처했다. 현재는 취약종단계에 등재되어있다. 또다른 아종인 애트워터초원멧닭은 한때 멕시코만 해변 뒤까지 해안 초원에 100만 마리가 살았다고 한다. 그러나 이들도 큰초원멧닭처럼 서식지파괴와 사냥 및 남획으로 1937년 이들에 관한 연구가 시작되었을 때, 이미 루이지애나주에서는 멸종했고, 텍사스주에서는 약 8,700마리로 감소했다. 30년후에는 개체수가 1,100마리도 남지 않아서 멸종위기종 목록에 들어갔다. 현재는 이 종역시 취약종으로 등급이 내려갔다. 애팔레치아 산맥 동쪽에서는 초원멧닭의 다른 아종이 뉴잉글랜드초원멧닭으로 불렸다.

---

[4] 1791년 일부 정치인들은 수렵시즌

In [ ]:
train_set_dir = "/data/ephemeral/home/data/train_dataset"
train_dataset = load_from_disk(train_set_dir)

In [44]:
print(len(train_dataset['train']['question']))

3952


In [ ]:
###
import tqdm

result_for_test = []

for i in tqdm.tqdm(range(len(train_dataset['train']['question']))):

    # 질문과 id
    test_q_query = train_dataset['train'][i]['question']
    test_q_id = train_dataset['train'][i]['id']

    # 골든리트리버 귀엽다
    retrieved_nodes_test = fusion_retriever.retrieve(test_q_query)


    # data for reranker
    docs_for_rerank_test = [n.node.text for n in retrieved_nodes_test]
    ids_for_rerank_test = [n.node.metadata['document_id'] for n in retrieved_nodes_test]


    # rerank result
    reranked_results_test = reranker.rerank(test_q_query, docs_for_rerank_test, ids_for_rerank_test, top_k=5)
    result_for_test.append([test_q_id, (list(np.array(reranked_results_test[1])[:,0].astype(int)))])
    
def convert_to_json(data):
    question_id = []
    document_list = []

    for q_id, doc_list in data:
        question_id.append(q_id)
        document_list.append(list(map(int, (doc_list))))
    result_dict = {
        "question_id": question_id,
        "document_id": document_list
    }
    return result_dict

json_test = convert_to_json(result_for_test)


file_path = './document_list/train_ner_hybrid.json'
with open(file_path, 'w') as f:
    json.dump(json_test, f)